#### ST449 Final Project

#### Connect4 Best Bots and Modifications

In [ ]:
# imports
import numpy as np
import sys
import pygame
import math
import random
import pandas as pd
from collections import namedtuple
# from aima_python_master.utils4e  import *
# from aima_python_master.games4e  import *


#### Generating the Board

In [ ]:
# ROW_COUNT = 6
# COLUMN_COUNT = 7

# def create_board(ROW_COUNT, COLUMN_COUNT):
# 	board = np.zeros((ROW_COUNT,COLUMN_COUNT), dtype= int)
# 	return board

# def drop_piece(board, row, col, piece):
# 	board[row][col] = piece

# def is_valid_location(board, col):
# 	return board[ROW_COUNT-1][col] == 0

# def get_next_open_row(board, col):
# 	for r in range(ROW_COUNT):
# 		if board[r][col] == 0:
# 			return r

# def print_board(board):
# 	print(np.flip(board, 0))

# def winning_move(board, piece):
# 	# Check horizontal locations for win
# 	for c in range(COLUMN_COUNT-3):
# 		for r in range(ROW_COUNT):
# 			if board[r][c] == piece and board[r][c+1] == piece and board[r][c+2] == piece and board[r][c+3] == piece:
# 				return True

# 	# Check vertical locations for win
# 	for c in range(COLUMN_COUNT):
# 		for r in range(ROW_COUNT-3):
# 			if board[r][c] == piece and board[r+1][c] == piece and board[r+2][c] == piece and board[r+3][c] == piece:
# 				return True

# 	# Check positively sloped diagonals
# 	for c in range(COLUMN_COUNT-3):
# 		for r in range(ROW_COUNT-3):
# 			if board[r][c] == piece and board[r+1][c+1] == piece and board[r+2][c+2] == piece and board[r+3][c+3] == piece:
# 				return True

# 	# Check negatively sloped diagonals
# 	for c in range(COLUMN_COUNT-3):
# 		for r in range(3, ROW_COUNT):
# 			if board[r][c] == piece and board[r-1][c+1] == piece and board[r-2][c+2] == piece and board[r-3][c+3] == piece:
# 				return True


# def generate_board_size(rows, columns):
# 	board = create_board(rows, columns)
# 	return board 

#### Players (Bots)

In [ ]:
def generate_segments(h=6, v=7, k=4):
    """ generate all segments of length k=4 on this board;
        segment is a list of lists of length 4 """
    segments = []

    # generate the vertical segments
    for y in range(1, v + 1):
        for x in range(1, h - k + 2):
            segment = []
            for t in range(k):
                segment.append((x + t, y))
            segments.append(segment)

    # generate the horizontal segments
    for x in range(1, h + 1):
        for y in range(1, v - k + 2):
            segment = []
            for t in range(k):
                segment.append((x, y + t))
            segments.append(segment)

    # generate the bottom left to top right diagonal segments
    for x in range(k, h + 1):
        for y in range(1, v - k + 2):
            segment = []
            for t in range(k):
                segment.append((x - t, y + t))
            segments.append(segment)

    # generate the top left to bottom right diagonal segments
    for y in range(1, v - k + 2):
        for x in range(1, h - k + 2):
            segment = []
            for t in range(k):
                segment.append((x + t, y + t))
            segments.append(segment)

    return segments

all_segments = generate_segments()

def count_in_segment(segment, state):
    """  Returns the count of 1's & 2's in a segment """
    """  Returns the count of X's & O's in a segment """
    X_count, O_count = 0, 0
    for x, y in segment:
        # if state[x-1][y-1] == 1:
        if state.board.get((x, y)) == 'X':
            X_count += 1
        # elif state[x-1][y-1] == 2:
        elif state.board.get((x, y)) == 'O':
            O_count += 1
    return X_count, O_count

def eval_segment(segment, state, player):
    """ Returns the evaluation score for a segment """
    X_count, O_count = count_in_segment(segment, state)
    if X_count > 0 and O_count > 0:
        return 0   # mixed segments are neutral

    count = max(X_count, O_count)
    score = 0

    if count == 1:  # open segments with 1 in a row (small chance)
        score = 1
    elif count == 2:  # open segments with 2 in a row (medium chance)
        score = 10
    elif count == 3:  # open segments with 3 in a row (big chance)
        score = 100
    elif count == 4:   # open segments with 4 in a row (game over)
        score = 100000

    if X_count > O_count:
        # dominant = 1
        dominant = 'X'
    else:
        # dominant = 2
        dominant = 'O'

    if dominant == player:
        return score
    else:
        return -score

def eval_fn(state, player):
    """ The evaluation function """
    total = 0
    for segment in all_segments:
        total += eval_segment(segment, state, player)
    return total


In [ ]:
# # Random bot
# def bot_move(board, piece):
#     valid_locations = [col for col in range(COLUMN_COUNT) if is_valid_location(board, col)]
#     col = random.choice(valid_locations)
#     row = get_next_open_row(board, col)
#     drop_piece(board, row, col, piece)

In [ ]:
# class Connect4Game:
#     def __init__(self, board, piece):
#         self.board = board
#         self.piece = piece

#     def to_move(self, state):
#         return self.piece

#     def actions(self, state):
#         return [col for col in range(COLUMN_COUNT) if is_valid_location(state, col)]

#     def result(self, state, action):
#         new_state = state.copy()
#         row = get_next_open_row(new_state, action)
#         drop_piece(new_state, row, action, self.piece)
#         return new_state

#     def terminal_test(self, state):
#         return winning_move(state, self.piece) or winning_move(state, 3 - self.piece) or len(self.actions(state)) == 0

#     def utility(self, state, player):
#         if winning_move(state, player):
#             return 100000
#         elif winning_move(state, 3 - player):
#             return -100000
#         else:
#             return eval_fn(state, player)
        


In [ ]:
def alpha_beta_cutoff_search(state, game, d=4, cutoff_test=None, eval_fn=None):
    """Search game to determine best action; use alpha-beta pruning.
    This version cuts off search and uses an evaluation function."""

    player = game.to_move(state)

    # Functions used by alpha_beta
    def max_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta, depth + 1))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta, depth + 1))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alpha_beta_cutoff_search starts here:
    # The default test cuts off at depth d or at a terminal state
    cutoff_test = (cutoff_test or (lambda state, depth: depth > d or game.terminal_test(state)))
    eval_fn = eval_fn or (lambda state, player: game.utility(state, player))
    best_score = -np.inf
    beta = np.inf
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta, 1)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action


# def alpha_beta_bot(board, piece):
#     game = Connect4Game(board, piece)
#     col = alpha_beta_cutoff_search(board, game)
#     row = get_next_open_row(board, col)
#     drop_piece(board, row, col, piece)

In [ ]:
# Monte Carlo Tree Search BOT

class MCT_Node:
    """Node in the Monte Carlo search tree, keeps track of the children states."""

    def __init__(self, parent=None, state=None, U=0, N=0):
        self.__dict__.update(parent=parent, state=state, U=U, N=N)
        self.children = {}
        self.actions = None


def ucb(n, C=1.4):
    return np.inf if n.N == 0 else n.U / n.N + C * np.sqrt(np.log(n.parent.N) / n.N)

def monte_carlo_tree_search(state, game, N=20000):
    def select(n):
        """select a leaf node in the tree"""
        if n.children:
            return select(max(n.children.keys(), key=ucb))
        else:
            return n

    def expand(n):
        """expand the leaf node by adding all its children states"""
        if not n.children and not game.terminal_test(n.state):
            n.children = {MCT_Node(state=game.result(n.state, action), parent=n): action
                          for action in game.actions(n.state)}
        return select(n)

    def simulate(game, state):
        """simulate the utility of current state by random picking a step"""
        player = game.to_move(state)
        while not game.terminal_test(state):
            action = random.choice(list(game.actions(state)))
            state = game.result(state, action)
        v = game.utility(state, player)
        return -v

    def backprop(n, utility):
        """passing the utility back to all parent nodes"""
        if utility > 0:
            n.U += utility
        # if utility == 0:
        #     n.U += 0.5
        n.N += 1
        if n.parent:
            backprop(n.parent, -utility)

    root = MCT_Node(state=state)

    for _ in range(N):
        leaf = select(root)
        child = expand(leaf)
        result = simulate(game, child.state)
        backprop(child, result)

    max_state = max(root.children, key=lambda p: p.N)

    return root.children.get(max_state)




# def monte_carlo_bot(board, piece):
#     game = Connect4Game(board, piece)
#     col = monte_carlo_tree_search(board, game)  # This should return the best action (column)
#     row = get_next_open_row(board, col)
#     drop_piece(board, row, col, piece)

#### Creating Game class (via seminar code)

In [ ]:
GameState = namedtuple('GameState', 'to_move, utility, board, moves')

class Game:
    """A game is similar to a problem, but it has a utility for each
    state and a terminal test instead of a path cost and a goal
    test. To create a game, subclass this class and implement actions,
    result, utility, and terminal_test. You may override display and
    successors or you can inherit their default methods. You will also
    need to set the .initial attribute to the initial state; this can
    be done in the constructor."""

    def actions(self, state):
        """Return a list of the allowable moves at this point."""
        raise NotImplementedError

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        raise NotImplementedError

    def utility(self, state, player):
        """Return the value of this final state to player."""
        raise NotImplementedError

    def terminal_test(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)

    def to_move(self, state):
        """Return the player whose move it is in this state."""
        return state.to_move

    def display(self, state):
        """Print or otherwise display the state."""
        print(state)

    def __repr__(self):
        return '<{}>'.format(self.__class__.__name__)

    def play_game(self, *players):
        """Play an n-person, move-alternating game."""
        state = self.initial
        while True:
            for player in players:
                move = player(self, state)
                state = self.result(state, move)
                if self.terminal_test(state):
                    self.display(state)
                    return self.utility(state, self.to_move(self.initial))
                    

In [ ]:
class C4(Game):
    """A TicTacToe-like game in which you can only make a move on the bottom
    row, or in a square directly above an occupied square. Traditionally
    played on a 6*7 board and requiring 4 in a row."""

    # def __init__(self, h=3, v=3, k=3):
    def __init__(self, h=6, v=7, k=4):
        self.h = h
        self.v = v
        self.k = k
        moves = [(x, y) for x in range(1, h + 1)
                 for y in range(1, v + 1)]
        self.initial = GameState(to_move='X', utility=0, board={}, moves=moves)

    def actions(self, state):
        # """Legal moves are any square not yet taken."""
        """ If we write (x, y) as the coordinate on the board,
        then the bottom row correspond to x=7, or equivalently x=self.h
        Recall that state.board is a dict and the keys are occupied locations. """
        # return state.moves
        return [(x, y) for (x, y) in state.moves
                if x == self.h or (x + 1 , y) in state.board]

    def result(self, state, move):
        if move not in state.moves:
            return state  # Illegal move has no effect
        board = state.board.copy()
        board[move] = state.to_move
        moves = list(state.moves)
        moves.remove(move)
        return GameState(to_move=('O' if state.to_move == 'X' else 'X'),
                         utility=self.compute_utility(board, move, state.to_move),
                         board=board, moves=moves)

    def utility(self, state, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return state.utility if player == 'X' else -state.utility

    def terminal_test(self, state):
        """A state is terminal if it is won or there are no empty squares."""
        return state.utility != 0 or len(state.moves) == 0

    def display(self, state):
        board = state.board
        for x in range(1, self.h + 1):
            for y in range(1, self.v + 1):
                print(board.get((x, y), '.'), end=' ')
            print()

    def compute_utility(self, board, move, player):
        """If 'X' wins with this move, return 1; if 'O' wins return -1; else return 0."""
        if (self.k_in_row(board, move, player, (0, 1)) or
                self.k_in_row(board, move, player, (1, 0)) or
                self.k_in_row(board, move, player, (1, -1)) or
                self.k_in_row(board, move, player, (1, 1))):
            return +1 if player == 'X' else -1
        else:
            return 0

    def k_in_row(self, board, move, player, delta_x_y):
        """Return true if there is a line through move on board for player."""
        (delta_x, delta_y) = delta_x_y
        x, y = move
        n = 0  # n is number of moves in row
        while board.get((x, y)) == player:
            n += 1
            x, y = x + delta_x, y + delta_y
        x, y = move
        while board.get((x, y)) == player:
            n += 1
            x, y = x - delta_x, y - delta_y
        n -= 1  # Because we counted move itself twice
        return n >= self.k
        

In [ ]:
testC4game = C4(h = 10, v = 10)
# testC4game = C4()

In [ ]:
# def alpha_beta_bot(board, piece):
#     game = Connect4Game(board, piece)
#     col = alpha_beta_cutoff_search(board, game)
#     row = get_next_open_row(board, col)
#     drop_piece(board, row, col, piece)

def test_MC_bot(game, state):
    return monte_carlo_tree_search(state, game, N = 1000)

def test_alpha_beta_bot(game, state):
    return alpha_beta_cutoff_search(state, game, d = 4)

def test_alpha_beta_eval_bot(game, state):
    return alpha_beta_cutoff_search(state, game, d = 5, eval_fn = eval_fn)

In [ ]:
testC4game.play_game(test_MC_bot, test_alpha_beta_bot)

#### Play Game

In [ ]:
# board = generate_board_size(6,7)
# game_over = False
# turn = 0

# while not game_over:
#     # Bot 1 move
#     if turn == 0:
#         monte_carlo_bot(board, 1)
#         if winning_move(board, 1):
#             print("Bot 1 wins!")
#             game_over = True

#     # Bot 2 move
#     else:
#         bot_move(board, 2)
#         if winning_move(board, 2):
#             print("Bot 2 wins!")
#             game_over = True

#     print_board(board)
#     turn += 1
#     turn = turn % 2


 # def play_game(self, *players):
 #        """Play an n-person, move-alternating game."""
 #        state = self.initial
 #        while True:
 #            for player in players:
 #                move = player(self, state)
 #                state = self.result(state, move)
 #                if self.terminal_test(state):
 #                    self.display(state)
 #                    return self.utility(state, self.to_move(self.initial))

In [ ]:
## Output shpould be matrix of how well each bot did in a number of n games against the other bots (first move randomly chosen)

#### Play Game (Different Rules)